## Patient Information Objects

### Patient

In [1]:
from models.patient import Patient

In [2]:
p1 = Patient('2712205f-755e-4897-acb3-926895b7d635')

In [3]:
p1.FIRST

'Emory494'

In [4]:
p1.get_age()

datetime.timedelta(days=29952, seconds=8212, microseconds=466776)

### Allergy

In [5]:
from models.allergy import Allergies

In [6]:
# A patient with multiple allergies
a1 = Allergies('df6b563d-1ff4-4833-9af8-84431e641e9c')

In [7]:
a1.allergies

In [8]:
# A patient with no allergies
a2 = Allergies('f0f3bc8d-ef38-49ce-a2bd-dfdda982b271')

In [9]:
a2.allergies

[]

by patient class

In [10]:
p1 = Patient('df6b563d-1ff4-4833-9af8-84431e641e9c')
p1.ALLERGY.get_n_allergies()

4

In [11]:
p2 = Patient('f0f3bc8d-ef38-49ce-a2bd-dfdda982b271')
p2.ALLERGY.get_n_allergies()

0

## Text generation